### **ETL** - *extract tranform load*

In [8]:
import os
import pandas as pd

In [11]:
DATA_FOLDER = ''
BIKE_RENTALS_FILE = 'tripdata_connected.csv'
WEATHER_FILE = 'weather_data.csv'

bike_rentals_df = pd.read_csv(os.path.join('..', DATA_FOLDER, BIKE_RENTALS_FILE))
weather_df = pd.read_csv(os.path.join('..', DATA_FOLDER, WEATHER_FILE),
                         encoding="Windows-1250", sep=";")

In [ ]:
bike_rentals_df.head()

,Ride_Id,Rideable_Type,Started_At,Ended_At,Start_Station_Name,Start_Station_Id,End_Station_Name,End_Station_Id,Start_Lat,Start_Lng,End_Lat,End_Lng,Member_Casual,Ride_Length,Day_Of_The_Week,Day
0,0BE9C131A5705D92,classic_bike,10/1/2021 0:00,10/1/2021 0:05,Damen Ave & Cortland St,13133,Winchester Ave & Elston Ave,KA1504000140,41.915983,-87.677335,41.924091,-87.676460,casual,0:05:00,6,Friday
1,ABA2BDC3595FC3E9,classic_bike,10/1/2021 0:00,10/1/2021 0:10,Morgan St & Lake St,TA1306000015,Noble St & Milwaukee Ave,13290,41.885483,-87.652305,41.900680,-87.662600,casual,0:10:00,6,Friday
2,74483AC18C8C6B90,classic_bike,10/1/2021 0:00,10/1/2021 0:08,Halsted St & Roscoe St,TA1309000025,Greenview Ave & Diversey Pkwy,13294,41.943670,-87.648950,41.932590,-87.665936,casual,0:08:00,6,Friday
3,A7711BCB74523614,electric_bike,10/1/2021 0:02,10/1/2021 0:03,NaN,NaN,NaN,NaN,41.930000,-87.700000,41.930000,-87.700000,casual,0:01:00,6,Friday
4,B0B9EB7622461EF4,classic_bike,10/1/2021 0:02,10/1/2021 0:17,MLK Jr Dr & 29th St,TA1307000139,Clinton St & Roosevelt Rd,WL-008,41.842052,-87.617000,41.867118,-87.641088,member,0:15:00,6,Friday


### **czas**

In [25]:
df_time = bike_rentals_df[['Ride_Id', 'Ride_Length', 'Started_At', 'Ended_At', 'Day']].copy()

df_time['Started_At'] = pd.to_datetime(df_time['Started_At'], format='%m/%d/%Y %H:%M')
df_time['Ended_At'] = pd.to_datetime(df_time['Ended_At'], format='%m/%d/%Y %H:%M')
df_time['Date'] = df_time['Started_At'].dt.date
df_time['Duration'] = (df_time['Ended_At'] - df_time['Started_At']).dt.total_seconds() // 60
df_time['Duration'].astype(int)
df_time['Time_Of_The_Day'] = df_time['Started_At'].dt.hour


def remap_time(time):
    if time >= 5 and time < 12:
        return 'Morning'
    elif time >= 12 and time < 18:
        return 'Afternoon'
    elif time >= 18 and time < 22:
        return 'Evening'
    else:
        return 'Night'
    
 
df_time['Time_Of_The_Day'] = df_time['Started_At']
df_time['Time_Of_The_Day'] = df_time['Time_Of_The_Day'].dt.hour.apply(remap_time)
display(df_time)
df_time.head()
df_time = df_time[['Ride_Id', 'Date', 'Duration', 'Day']].copy()
df_time.head()

,Ride_Id,Ride_Length,Started_At,Ended_At,Day,Date,Duration,Time_Of_The_Day
0,0BE9C131A5705D92,0:05:00,2021-10-01 00:00:00,2021-10-01 00:05:00,Friday,2021-10-01,5.0,Night
1,ABA2BDC3595FC3E9,0:10:00,2021-10-01 00:00:00,2021-10-01 00:10:00,Friday,2021-10-01,10.0,Night
2,74483AC18C8C6B90,0:08:00,2021-10-01 00:00:00,2021-10-01 00:08:00,Friday,2021-10-01,8.0,Night
3,A7711BCB74523614,0:01:00,2021-10-01 00:02:00,2021-10-01 00:03:00,Friday,2021-10-01,1.0,Night
4,B0B9EB7622461EF4,0:15:00,2021-10-01 00:02:00,2021-10-01 00:17:00,Friday,2021-10-01,15.0,Night
...,...,...,...,...,...,...,...,...
5138708,72C70C6F373CE503,0:07:00,2022-09-30 23:59:00,2022-10-01 00:06:00,Friday,2022-09-30,7.0,Night
5138709,D27A944387D7C4EC,0:23:00,2022-09-30 23:59:00,2022-10-01 00:22:00,Friday,2022-09-30,23.0,Night
5138710,99EC44F3C28B0597,0:09:00,2022-09-30 23:59:00,2022-10-01 00:08:00,Friday,2022-09-30,9.0,Night
5138711,58F0790EB07E1E40,0:08:00,2022-09-30 23:59:00,2022-10-01 00:07:00,Friday,2022-09-30,8.0,Night


,Ride_Id,Date,Duration,Day
0,0BE9C131A5705D92,2021-10-01,5.0,Friday
1,ABA2BDC3595FC3E9,2021-10-01,10.0,Friday
2,74483AC18C8C6B90,2021-10-01,8.0,Friday
3,A7711BCB74523614,2021-10-01,1.0,Friday
4,B0B9EB7622461EF4,2021-10-01,15.0,Friday


### **lokalizacja**

In [ ]:
df_location = bike_rentals_df[['Ride_Id', 'Start_Station_Id', 'End_Station_Id']].copy()
df_stations = bike_rentals_df[['Start_Station_Id', 'Start_Station_Name', 'Start_Lat', 'Start_Lng']].copy()
len(df_stations['Start_Station_Id'].unique()), len(df_stations)

(1279, 5138713)

In [ ]:
df_stations = df_stations.drop_duplicates(subset=['Start_Station_Id'])
df_stations.head()

,Start_Station_Id,Start_Station_Name,Start_Lat,Start_Lng
0,13133,Damen Ave & Cortland St,41.915983,-87.677335
1,TA1306000015,Morgan St & Lake St,41.885483,-87.652305
2,TA1309000025,Halsted St & Roscoe St,41.943670,-87.648950
3,NaN,NaN,41.930000,-87.700000
4,TA1307000139,MLK Jr Dr & 29th St,41.842052,-87.617000


### **typy**

In [ ]:
df_types = bike_rentals_df[['Ride_Id', 'Rideable_Type', 'Member_Casual']].copy()
df_types.head()

,Ride_Id,Rideable_Type,Member_Casual
0,0BE9C131A5705D92,classic_bike,casual
1,ABA2BDC3595FC3E9,classic_bike,casual
2,74483AC18C8C6B90,classic_bike,casual
3,A7711BCB74523614,electric_bike,casual
4,B0B9EB7622461EF4,classic_bike,member


### **pogoda**

In [ ]:
weather_df.head()

,time,temperature_2m (°C),precipitation (mm),cloudcover (%),windspeed_10m (km/h),latitude,longitude,elevation,utc_offset_seconds,timezone,timezone_abbreviation
0,2021-10-01T00:00,"19,3",0,23.0,"9,4","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
1,2021-10-01T01:00,"18,8",0,14.0,"9,9","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
2,2021-10-01T02:00,"18,5",0,4.0,"9,6","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
3,2021-10-01T03:00,"18,1",0,0.0,"9,7","41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT
4,2021-10-01T04:00,"17,8",0,0.0,10,"41,800003","-87,6",179.0,-18000.0,America/Chicago,CDT


In [35]:
df_weather = pd.DataFrame()
weather_cols = ['Time', 'Temperature', 'Precipitation', 'Wind_Speed', 'Cloudcover', 'Latitude', 'Longitude']
df_weather[weather_cols] = weather_df[[
    'time', 'temperature_2m (°C)', 'precipitation (mm)', 'windspeed_10m (km/h)', 'cloudcover (%)', 'latitude', 'longitude'
    ]].copy()

In [36]:
df_weather['Time'] = pd.to_datetime(df_weather['Time'], format='%Y-%m-%d %H:%M:%S')

float_cols = ['Temperature', 'Precipitation', 'Wind_Speed', 'Cloudcover', 'Latitude', 'Longitude']
df_weather[float_cols] = df_weather[float_cols].replace(',', '.', regex=True).astype(float)

df_weather['Date'] = df_weather['Time'].dt.date

df_weather.head(8)

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,19.3,0.0,9.4,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,18.8,0.0,9.9,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,18.5,0.0,9.6,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,18.1,0.0,9.7,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,17.8,0.0,10.0,0.0,41.800003,-87.6,2021-10-01
5,2021-10-01 05:00:00,17.5,0.0,10.2,0.0,41.800003,-87.6,2021-10-01
6,2021-10-01 06:00:00,17.3,0.0,10.7,0.0,41.800003,-87.6,2021-10-01
7,2021-10-01 07:00:00,17.1,0.0,10.5,0.0,41.800003,-87.6,2021-10-01


In [37]:
def remap_perciption(perciption):
    if perciption == 0:
        return 'No'
    elif perciption < 2.5:
        return 'Light'
    elif perciption < 7.6:
        return 'Moderate'
    elif perciption < 50.8:
        return 'Heavy'
    else:
        return 'Violent'
    
df_weather['Precipitation'] = df_weather['Precipitation'].apply(remap_perciption)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,19.3,No,9.4,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,18.8,No,9.9,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,18.5,No,9.6,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,18.1,No,9.7,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,17.8,No,10.0,0.0,41.800003,-87.6,2021-10-01


In [38]:
def remap_wind(wind):
    if wind < 1.6:
        return 'Calm'
    elif wind < 5.5:
        return 'Light'
    elif wind < 11.1:
        return 'Moderate'
    elif wind < 19.8:
        return 'Fresh'
    elif wind < 28.5:
        return 'Strong'
    elif wind < 38.9:
        return 'Near Gale'
    elif wind < 49.6:
        return 'Gale'
    elif wind < 61.2:
        return 'Strong Gale'
    elif wind < 74.2:
        return 'Storm'
    else:
        return 'Violent Storm'

df_weather['Wind_Speed'] = df_weather['Wind_Speed'].apply(remap_wind)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,19.3,No,Moderate,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,18.8,No,Moderate,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,18.5,No,Moderate,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,18.1,No,Moderate,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,17.8,No,Moderate,0.0,41.800003,-87.6,2021-10-01


In [39]:
df_weather['Wind_Speed'].value_counts()

Fresh            3668
Moderate         2391
Strong           1768
Light             496
Near Gale         351
Gale               46
Calm               40
Violent Storm       3
Name: Wind_Speed, dtype: int64

In [40]:
def remap_temperature(temperature):
    if temperature < -20:
        return 'Extreme Cold'
    elif temperature < -10:
        return 'Very Cold'
    elif temperature < 0:
        return 'Cold'
    elif temperature < 10:
        return 'Cool'
    elif temperature < 20:
        return 'Mild'
    elif temperature < 30:
        return 'Warm'
    elif temperature < 40:
        return 'Hot'
    else:
        return 'Very Hot'

df_weather['Temperature'] = df_weather['Temperature'].apply(remap_temperature)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,Mild,No,Moderate,23.0,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,Mild,No,Moderate,14.0,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,Mild,No,Moderate,4.0,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,Mild,No,Moderate,0.0,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,Mild,No,Moderate,0.0,41.800003,-87.6,2021-10-01


In [41]:
df_weather['Temperature'].value_counts()

Cool         2614
Mild         2334
Warm         2277
Cold         1316
Very Cold     189
Hot            30
Very Hot        3
Name: Temperature, dtype: int64

In [44]:
def remap_cloudcover(cloud_cover):
    if cloud_cover == 0:
        return 'No'
    elif cloud_cover > 0 and cloud_cover < 10:
        return 'Few'
    elif cloud_cover >=10 and cloud_cover < 25:
        return 'Isolated'
    elif cloud_cover >=25 and cloud_cover < 50:
        return 'Scattered'
    elif cloud_cover >=50 and cloud_cover < 90:
        return 'Broken'
    else:  
        return 'Overcast'


df_weather['Cloudcover'] = df_weather['Cloudcover'].apply(remap_cloudcover)
df_weather.head()

,Time,Temperature,Precipitation,Wind_Speed,Cloudcover,Latitude,Longitude,Date
0,2021-10-01 00:00:00,Mild,No,Moderate,Isolated,41.800003,-87.6,2021-10-01
1,2021-10-01 01:00:00,Mild,No,Moderate,Isolated,41.800003,-87.6,2021-10-01
2,2021-10-01 02:00:00,Mild,No,Moderate,Few,41.800003,-87.6,2021-10-01
3,2021-10-01 03:00:00,Mild,No,Moderate,No,41.800003,-87.6,2021-10-01
4,2021-10-01 04:00:00,Mild,No,Moderate,No,41.800003,-87.6,2021-10-01
